This is an example of what you might do when you first look at some root files that you've never seen before. In this exercise, we will imagine that we're just getting started on some sort of dijet resonance analysis. Luckily, your signal, backgrounds, and data have all been ntuplized already, and the output ntuples are located on eos at: 

`/eos/uscms/store/user/hats/PyRoot/2017/qcd_samples`

In Jupyter, we can execute shell commands by prepending the usual commands with `!`

In [ ]:
!eos root://cmseos.fnal.gov/ ls /store/user/hats/PyRoot/2017/qcd_samples

Let's see our signal samples, which are in `shuffled_sig`:

In [ ]:
!eos root://cmseos.fnal.gov/ ls /store/user/hats/PyRoot/2017/qcd_samples/shuffled_sig

Let's use pyROOT to take a look at the basic structure of the file and the branches therein.

> Note:
>   One could do all of this in an interactive python session, using 
>
>   `python -i`
>
>  Interactive pyROOT is very useful for quickly looking at ROOT files and other tasks, such as formatting ROOT graphics. If you are interested in this, please ask the instructor for a demonstration of where this can come in handy (e.g. reformatting plots in interactive pyROOT while a TCanvas is open).

Commonly, pyROOT scripts start out with

`from ROOT import *`

but this is not allowed in JupyROOT. However, we can still save some typing by referring to ROOT with a shorter name:

In [ ]:
import ROOT as r

In [ ]:
firstLookFile = r.TFile.Open("root://cmseos.fnal.gov//store/user/hats/PyRoot/2017/qcd_samples/shuffled_sig/sig_shuffled_0.root")

In [ ]:
firstLookFile.ls()

In [ ]:
firstLookTree = firstLookFile.Get("tree")

Already, we're benefitting from pyROOT's syntax. Note that a similar step in an interactive ROOT session would require a bit more effort:

`TTree* firstLookTree = (TTree*) firstLookFile->Get("tree")`

Let's use a quick python loop to list all the branches in our tree.

In [ ]:
for branch in firstLookTree.GetListOfBranches():
    print branch.GetName()

Our analysis is going to look for dijet resonances using AK4 jets. So to get a feel for the data, let's look at what the AK4 jet pT spectrum looks like.

In [ ]:
can = r.TCanvas()      
firstLookTree.Draw("jetAK4_pt")
can.Draw()

> Note that in interactive pyROOT, one can simply call `firstLookTree.Draw("jetAK4_pt")` without bothering to explicitly instantiate a TCanvas and call TCanvas.Draw() on it

Let's do some quick changes to the plot. `r.gStyle` gives us access to common style changes for ROOT graphics.

In [ ]:
r.gStyle.SetOptStat(0)
can.SetLogy()
can.Draw()

Let's quickly demonstrate python's introspection features. ROOT automatically builds a lot of things into a TCanvas that might not be obvious when you simply call `TTree.Draw('branchName')`, so let's see what's in our canvas and what methods we could call on it. And let's make the output more readable while we're at it. 

In [ ]:
import inspect
from pprint import pprint
for primitive in can.GetListOfPrimitives():
    print primitive, pprint(inspect.getmembers(primitive, predicate=inspect.ismethod))

Now that we know what's in our trees and gotten a quick look to sanity-check the data, we'll get started trying to tackle the many gigs of data that are at hand. pyROOT is not the right tool for crunching this much data, so let's move on to preparing a C++ class to do the number crunching.

In [ ]:
firstLookTree.MakeClass("hatsTrees")

Let's see what this made for us:

In [ ]:
!ls -lrth

Our aim is to use this C++ class to do the CPU-intensive work of our analysis, but to keep it as a "plugin" inside our pyroot framework.

>Please move on to the notebook `prepareCppClass.ipynb`